### Install the Python Modules

In [1]:
!pip install google-generativeai
!pip install langchain
!pip install pypdf
!pip install jupyter
!pip install InstructorEmbedding
!pip install transformers
!pip install huggingface
!pip install torch
!pip install sentence_transformers

### Code

In [1]:
from langchain.llms import GooglePalm

llm = GooglePalm(google_api_key='', temperature=0.7)

In [2]:
# ai_intro = llm("Write about the Artificial Intelligence in 2023")
# print(ai_intro)

In [3]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='codebasics_faqs.csv', source_column='prompt')
data = loader.load()

In [4]:
import torch

# Before starting your training loop
torch.cuda.empty_cache()

In [5]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS

instructor_embeddings = HuggingFaceInstructEmbeddings()

vectordb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)


load INSTRUCTOR_Transformer
max_seq_length  512


In [6]:
retriever = vectordb.as_retriever()

rdocs = retriever.get_relevant_documents("how about job placement support")
print(rdocs)

[Document(page_content='prompt: What career services are available for graduating students?\nresponse: FAST-NUCES provides comprehensive career services for graduating students, including resume building, interview preparation, and job placement assistance. The university has strong connections with leading employers.', metadata={'source': 'What career services are available for graduating students?', 'row': 21}), Document(page_content="prompt: Are there any specific internship programs for Computer Science students?\nresponse: Yes, FAST-NUCES facilitates internships for Computer Science students through collaborations with industry partners. The university's career services office assists students in securing relevant internships.", metadata={'source': 'Are there any specific internship programs for Computer Science students?', 'row': 3}), Document(page_content='prompt: What support does FAST-NUCES provide for students with entrepreneurial ideas?\nresponse: FAST-NUCES supports student

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [8]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm, 
            chain_type="stuff",
            retriever=retriever,
            input_key="query",
            return_source_documents=True,
            chain_type_kwargs={"prompt": PROMPT})

In [9]:
chain("do you have a plain of launching blockchain course")

{'query': 'do you have a plain of launching blockchain course',
 'result': " I don't know.",
 'source_documents': [Document(page_content='prompt: Is there a mentorship program for incoming students?\nresponse: Yes, FAST-NUCES has a mentorship program to support incoming students. New students are paired with experienced mentors who provide guidance, advice, and assistance with the academic transition.', metadata={'source': 'Is there a mentorship program for incoming students?', 'row': 15}),
  Document(page_content='prompt: Describe the startup ecosystem at FAST-NUCES.\nresponse: FAST-NUCES has a thriving startup ecosystem supported by dedicated entrepreneurship centers. Students with entrepreneurial aspirations can access mentorship, funding, and resources to launch their startups.', metadata={'source': 'Describe the startup ecosystem at FAST-NUCES.', 'row': 9}),
  Document(page_content="prompt: Are there any specific internship programs for Computer Science students?\nresponse: Yes, F